# December 19, 2023

https://adventofcode.com/2023/day/19

In [ ]:
import re

In [9]:
test = f'''px{{a<2006:qkq,m>2090:A,rfg}}
pv{{a>1716:R,A}}
lnx{{m>1548:A,A}}
rfg{{s<537:gd,x>2440:R,A}}
qs{{s>3448:A,lnx}}
qkq{{x<1416:A,crn}}
crn{{x>2662:A,R}}
in{{s<1351:px,qqz}}
qqz{{s>2770:qs,m<1801:hdj,R}}
gd{{a>3333:R,R}}
hdj{{m>838:A,pv}}

{{x=787,m=2655,a=1222,s=2876}}
{{x=1679,m=44,a=2067,s=496}}
{{x=2036,m=264,a=79,s=2244}}
{{x=2461,m=1339,a=466,s=291}}
{{x=2127,m=1623,a=2188,s=1013}}'''

test_text = test.split("\n")

In [10]:
test_text

['px{a<2006:qkq,m>2090:A,rfg}',
 'pv{a>1716:R,A}',
 'lnx{m>1548:A,A}',
 'rfg{s<537:gd,x>2440:R,A}',
 'qs{s>3448:A,lnx}',
 'qkq{x<1416:A,crn}',
 'crn{x>2662:A,R}',
 'in{s<1351:px,qqz}',
 'qqz{s>2770:qs,m<1801:hdj,R}',
 'gd{a>3333:R,R}',
 'hdj{m>838:A,pv}',
 '',
 '{x=787,m=2655,a=1222,s=2876}',
 '{x=1679,m=44,a=2067,s=496}',
 '{x=2036,m=264,a=79,s=2244}',
 '{x=2461,m=1339,a=466,s=291}',
 '{x=2127,m=1623,a=2188,s=1013}']

In [11]:
fn = "data/19.txt"
with open(fn, "r") as file:
    text = file.readlines()

puzz_text = [x.strip() for x in text]

In [99]:
class Test:
    '''Test class takes a spec string and then can test toys using test method.'''
    def __init__(self, spec):
        if ":" in spec:
            m = re.fullmatch("([xmas])(.)(\d+):(\w+)", spec)
            self.dim = m[1]
            self.op = m[2]
            self.cutoff = int(m[3])
            self.out = m[4]
        else:
            self.dim = self.op = self.cutoff = None
            self.out = spec


    def test(self, toy):
        if self.op is None:
            return self.out
        elif self.op == ">":
            if toy[self.dim] > self.cutoff:
                return self.out
            else:
                return None
        elif self.op == "<":
            if toy[self.dim] < self.cutoff:
                return self.out
            else:
                return None
        else:
            raise Exception("Unrecognized operation: " + m[2])

In [100]:
class Workflow:
    '''create a named workflow with a method to run a toy through it'''
    def __init__(self, line):
        m = re.fullmatch("(\w+){(.*)}", line)
        self.name = m[1]
        specs = m[2].split(",")
        
        self.tests = [Test(spec) for spec in specs]

    def run(self, toy):
        for test in self.tests:
            out = test.test(toy)
            if out is not None:
                return out

### Part 1

In [101]:
def parse_input(text):
    workflows = {}
    toys = []
    for i, line in enumerate(text):
        if len(line) == 0:
            break

        wf = Workflow(line)
        workflows[wf.name] = wf
    
    for line in text[i+1:]:
        m = re.fullmatch("{x=(\d+),m=(\d+),a=(\d+),s=(\d+)}", line)
        toy = {'x':int(m[1]), 'm':int(m[2]), 'a':int(m[3]), 's':int(m[4])}
        toys.append(toy)

    return {'workflows':workflows, 'toys':toys}

def sort_toy( workflows, toy ):
    out = workflows['in'].run(toy)
    while out not in ('A', 'R'):
        out = workflows[out].run(toy)

    if out == 'A':
        return True
    elif out == 'R':
        return False
    else:
        raise Exception("toy sort fail")

def part1( puzz, verbose=False ):
    tot = 0
    for toy in puzz['toys']:
        accept = sort_toy( puzz['workflows'], toy )
        if verbose:
            print(accept)
        if accept:
            tot += toy['x'] + toy['m'] + toy['a'] + toy['s']
            if verbose:
                print(tot)
    
    return tot


In [102]:
test = parse_input(test_text)

In [103]:
sort_toy(test['workflows'], test['toys'][0])

True

In [104]:
part1(test, verbose=True)

True
7540
False
True
12163
False
True
19114


19114

In [105]:
puzz = parse_input(puzz_text)
part1(puzz)

348378

### Part 2

In [106]:
test_text

['px{a<2006:qkq,m>2090:A,rfg}',
 'pv{a>1716:R,A}',
 'lnx{m>1548:A,A}',
 'rfg{s<537:gd,x>2440:R,A}',
 'qs{s>3448:A,lnx}',
 'qkq{x<1416:A,crn}',
 'crn{x>2662:A,R}',
 'in{s<1351:px,qqz}',
 'qqz{s>2770:qs,m<1801:hdj,R}',
 'gd{a>3333:R,R}',
 'hdj{m>838:A,pv}',
 '',
 '{x=787,m=2655,a=1222,s=2876}',
 '{x=1679,m=44,a=2067,s=496}',
 '{x=2036,m=264,a=79,s=2244}',
 '{x=2461,m=1339,a=466,s=291}',
 '{x=2127,m=1623,a=2188,s=1013}']

In [165]:
def copy_toy_range( tr ):
    return {'flow':tr['flow'], 'x':tr['x'].copy(), 'm':tr['m'].copy(),'a':tr['a'].copy(), 's':tr['s'].copy()}

def split_toy_range( wf, tr ):
    resolved = [] # keep track of which toy_ranges have their next workflow determined

    for t in wf.tests:
        dim = t.dim
        cutoff = t.cutoff
        op = t.op
        out = t.out

        #print(wf.name, dim, op, cutoff, out)

        if op is None:
            # reached default case
            tr['flow'] = out
            resolved.append(tr)
            return resolved

        dmin = tr[dim][0]
        dmax = tr[dim][1]

        if op == ">":
            if dmin > cutoff:
                # all remaining toys pass this test and receive same output
                tr['flow'] = out
                resolved.append(tr)
                return resolved

            # all toys with dim > cutoff are resolved
            tr_resolved = copy_toy_range(tr)
            tr_resolved[dim][0] = cutoff+1
            tr_resolved['flow'] = out
            resolved.append(tr_resolved)

            # remaining toyrange to resolve has dim <= cutoff
            tr[dim][1] = cutoff

        elif op == "<":
            if dmax < cutoff:
                # all remaining toys pass this test and receive same output
                tr['flow'] = out
                resolved.append(tr)
                return resolved
                
            # all toys with dim < cutoff are resolved
            tr_resolved = copy_toy_range(tr)
            tr_resolved[dim][1] = cutoff-1
            tr_resolved['flow'] = out
            resolved.append(tr_resolved)

            # remaining toyrange to resolve has dim >= cutoff
            tr[dim][0] = cutoff
        
        #print(tr)
        #print(resolved)
    raise Exception("You shouldn't be here")


def analyze_toy_ranges( puzz, toy_ranges ):
    accept = []
    reject = []
    unresolved = []

    while len(toy_ranges) > 0:
        #print("\n------\n")
        #print(toy_ranges)
        results = split_toy_range( puzz['workflows'][toy_ranges[0]['flow']], toy_ranges[0] )

        #print(results)
        for res in results:

            if res['flow'] == 'A':
                accept.append( res )
            elif res['flow'] == 'R':
                reject.append( res )
            else:
                unresolved.append( res )

        toy_ranges = unresolved + toy_ranges[1:]
        unresolved = []

    return accept, reject

        
def part2( puzz ):
    toy_ranges = [ {'flow':'in', 'x':[1,4000], 'm':[1,4000], 'a':[1,4000], 's':[1,4000]} ]
    acc, rej = analyze_toy_ranges( puzz, toy_ranges )

    tot = 0
    for tr in acc:
        ntoys = (tr['x'][1] - tr['x'][0] + 1)*(tr['m'][1] - tr['m'][0] + 1)*(tr['a'][1] - tr['a'][0] + 1)*(tr['s'][1] - tr['s'][0] + 1)
        tot += ntoys
    return tot




In [159]:
toy_ranges = [ {'flow':'in', 'x':[1,4000], 'm':[1,4000], 'a':[1,4000], 's':[1,4000]} ]
acc, rej = analyze_toy_ranges( test, toy_ranges )

In [160]:
acc

[{'flow': 'A',
  'x': [1, 4000],
  'm': [2091, 4000],
  'a': [2006, 4000],
  's': [1, 1350]},
 {'flow': 'A', 'x': [1, 1415], 'm': [1, 4000], 'a': [1, 2005], 's': [1, 1350]},
 {'flow': 'A',
  'x': [2663, 4000],
  'm': [1, 4000],
  'a': [1, 2005],
  's': [1, 1350]},
 {'flow': 'A',
  'x': [1, 2440],
  'm': [1, 2090],
  'a': [2006, 4000],
  's': [537, 1350]},
 {'flow': 'A',
  'x': [1, 4000],
  'm': [1, 4000],
  'a': [1, 4000],
  's': [3449, 4000]},
 {'flow': 'A',
  'x': [1, 4000],
  'm': [1549, 4000],
  'a': [1, 4000],
  's': [2771, 3448]},
 {'flow': 'A',
  'x': [1, 4000],
  'm': [1, 1548],
  'a': [1, 4000],
  's': [2771, 3448]},
 {'flow': 'A',
  'x': [1, 4000],
  'm': [839, 1800],
  'a': [1, 4000],
  's': [1351, 2770]},
 {'flow': 'A',
  'x': [1, 4000],
  'm': [1, 838],
  'a': [1, 1716],
  's': [1351, 2770]}]

In [161]:
tot = 0
for tr in acc:
    ntoys = (tr['x'][1] - tr['x'][0] + 1)*(tr['m'][1] - tr['m'][0] + 1)*(tr['a'][1] - tr['a'][0] + 1)*(tr['s'][1] - tr['s'][0] + 1)
    tot += ntoys
tot

167409079868000

In [157]:
tot == 167409079868000

True

In [166]:
part2( test )

167409079868000

In [167]:
part2( puzz )

121158073425385